In [ ]:
from pyvis.network import Network
import pandas as pd

# PyViz

In [ ]:
df = pd.read_excel("employer_matching.xlsx", index_col=0)
nodes = df.columns.to_list()
nodes = [node.strip() for node in nodes]

In [ ]:
import pandas as pd

df = pd.read_excel("employer_matching.xlsx", index_col=0)
nodes = df.columns.to_list()
nodes = [node.strip() for node in nodes]
nodes

In [ ]:
def get_edges(node: str, weights: list, all_nodes: list, minium_weight: int):

    nodes = all_nodes.copy()

    # Remove target node
    nodes.remove(node)

    # Create a list of edges with weights
    edges = [(node, connection, weight) for connection, weight in zip(nodes, weights)]

    # Get only edges with weights greater than the minimum weight
    edges = [edge for edge in edges if edge[2] >= minium_weight]

    return edges

def draw_network(
    nodes: list,
    df: pd.DataFrame,
    minium_weight: int = 0,
    repulsion: int = 100,
    spring_length=200,
    buttons=False,
):

    net = Network("500px", "500px", notebook=True)
    net.add_nodes(nodes)

    # add edges
    for node, weights in df.iterrows():
        edges = get_edges(node, weights, nodes, minium_weight)
        net.add_edges(edges)

    # change node distance and spring length
    net.repulsion(repulsion, spring_length=spring_length)

    # Tweek configuration UI
    net.show_buttons(filter_=buttons)
    return net

In [ ]:
net = draw_network(nodes, df, repulsion=100, spring_length=500)
net.show("match.html")